In [1]:
!pip install rouge
from rouge import Rouge 
import pandas as pd
import numpy as np

In [2]:
import pandas as pd

path_file= '/content/drive/My Drive/Colab Notebooks/data/bartlageCNN_predicted_sum.csv'
df = pd.read_csv(path_file)

#drop row with nan in predicted highlights or reference highlights
df = df[df['shorten_highlights'].notna()]
df = df[df['predicted_highlights'].notna()]

# reindex 
df = df.reset_index(drop=True)

print("shape after drop NaN: ",df.shape)
# df.loc[i,'predicted_highlights']
df.head(5)

shape after drop NaN:  (9999, 8)


,Unnamed: 0,Unnamed: 0.1,story,highlights,id,shorten_story,shorten_highlights,predicted_highlights
0,0,0,['ever hopped in a taxi and sat in traffic for...,['mellowcab is a threewheeled vehicle that the...,id_cnn_88698,ever hopped in a taxi and sat in traffic for h...,mellowcab is a threewheeled vehicle that the o...,m mellowcabs electric threewheeled vehicles de...
1,1,1,['a mother of six has claimed she was threaten...,['amanda moss and husband simon booked family ...,id_dm_26727,a mother of six has claimed she was threatened...,amanda moss and husband simon booked family ho...,amanda moss of liverpool wanted to take their ...
2,2,2,['thomas morris who was strangled by his own k...,['thomas morris was found dead in home gym in ...,id_dm_205431,thomas morris who was strangled by his own kar...,thomas morris was found dead in home gym in mo...,fitness fanatic accidentally strangled himself...
3,3,3,['britains princess anne and major winner anni...,"['ra announces first women members', 'princess...",id_cnn_47838,britains princess anne and major winner annika...,ra announces first women members,princess anne and major winner annika sorensta...
4,4,4,['it is alleged that michael karkoc was an off...,['alleged michael karkoc was in the ssled ukra...,id_dm_63684,it is alleged that michael karkoc was an offic...,alleged michael karkoc was in the ssled ukrani...,michael karkoc was an officer in the ssled ukr...


In [3]:
# get highlight, pred highlight, id to lists for ROUGE calculation
list_pred = []
list_ref =[]
list_id = []

for i in range(0,len(df)):
  list_pred.append(df.loc[i,'predicted_highlights'])
  list_ref.append(df.loc[i,'shorten_highlights'])
  list_id.append(df.loc[i,'id'])
# list_pred, list_ref

In [4]:
# calculate with ROUGE metric
rouge = Rouge()
scores = rouge.get_scores(list_pred, list_ref)



In [5]:
df_scores = pd.DataFrame(scores)
df_scores['id'] = list_id
df_scores

,rouge-1,rouge-2,rouge-l,id
0,"{'f': 0.1379310294887041, 'p': 0.1428571428571...","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.14285713785714302, 'p': 0.142857142857...",id_cnn_88698
1,"{'f': 0.25806451175858486, 'p': 0.190476190476...","{'f': 0.13793103020214045, 'p': 0.1, 'r': 0.22...","{'f': 0.27586206444708683, 'p': 0.210526315789...",id_dm_26727
2,"{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.0, 'p': 0.0, 'r': 0.0}",id_dm_205431
3,"{'f': 0.23076922766272187, 'p': 0.142857142857...","{'f': 0.16666666388888893, 'p': 0.1, 'r': 0.5}","{'f': 0.26086956181474485, 'p': 0.166666666666...",id_cnn_47838
4,"{'f': 0.6896551677051131, 'p': 0.5555555555555...","{'f': 0.5925925879286695, 'p': 0.4705882352941...","{'f': 0.7142857095153062, 'p': 0.5882352941176...",id_dm_63684
...,...,...,...,...
9994,"{'f': 0.07407406913580279, 'p': 0.066666666666...","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.07407406913580279, 'p': 0.066666666666...",id_dm_183675
9995,"{'f': 0.370370365925926, 'p': 0.27777777777777...","{'f': 0.23999999564800004, 'p': 0.176470588235...","{'f': 0.370370365925926, 'p': 0.27777777777777...",id_dm_14557
9996,"{'f': 0.4615384570940172, 'p': 0.3461538461538...","{'f': 0.3783783739956173, 'p': 0.28, 'r': 0.58...","{'f': 0.4864864819284149, 'p': 0.375, 'r': 0.6...",id_cnn_90018
9997,"{'f': 0.30303029825528016, 'p': 0.25, 'r': 0.3...","{'f': 0.12903225331945908, 'p': 0.105263157894...","{'f': 0.2499999951757813, 'p': 0.2105263157894...",id_dm_192809


In [6]:
# create new column names
list_rouge = list(df_scores)
list_rouge = [rouge.upper() for rouge in list_rouge]
list_metric = ['F1','Precision','Recall']

list_col_name = []

for rouge in list_rouge:
  for metric in list_metric:
    list_col_name.append(rouge+'_'+metric)

list_col_name

['ROUGE-1_F1',
 'ROUGE-1_Precision',
 'ROUGE-1_Recall',
 'ROUGE-2_F1',
 'ROUGE-2_Precision',
 'ROUGE-2_Recall',
 'ROUGE-L_F1',
 'ROUGE-L_Precision',
 'ROUGE-L_Recall',
 'ID_F1',
 'ID_Precision',
 'ID_Recall']

In [7]:
# create new columns
for col in list_col_name:
  # if column not exist, create new one
  if col not in df_scores:
    df_scores[col] = '-'
df_scores.head(5)

,rouge-1,rouge-2,rouge-l,id,ROUGE-1_F1,ROUGE-1_Precision,ROUGE-1_Recall,ROUGE-2_F1,ROUGE-2_Precision,ROUGE-2_Recall,ROUGE-L_F1,ROUGE-L_Precision,ROUGE-L_Recall,ID_F1,ID_Precision,ID_Recall
0,"{'f': 0.1379310294887041, 'p': 0.1428571428571...","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.14285713785714302, 'p': 0.142857142857...",id_cnn_88698,-,-,-,-,-,-,-,-,-,-,-,-
1,"{'f': 0.25806451175858486, 'p': 0.190476190476...","{'f': 0.13793103020214045, 'p': 0.1, 'r': 0.22...","{'f': 0.27586206444708683, 'p': 0.210526315789...",id_dm_26727,-,-,-,-,-,-,-,-,-,-,-,-
2,"{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.0, 'p': 0.0, 'r': 0.0}",id_dm_205431,-,-,-,-,-,-,-,-,-,-,-,-
3,"{'f': 0.23076922766272187, 'p': 0.142857142857...","{'f': 0.16666666388888893, 'p': 0.1, 'r': 0.5}","{'f': 0.26086956181474485, 'p': 0.166666666666...",id_cnn_47838,-,-,-,-,-,-,-,-,-,-,-,-
4,"{'f': 0.6896551677051131, 'p': 0.5555555555555...","{'f': 0.5925925879286695, 'p': 0.4705882352941...","{'f': 0.7142857095153062, 'p': 0.5882352941176...",id_dm_63684,-,-,-,-,-,-,-,-,-,-,-,-


In [8]:
dict_metric_name = {'r':'Recall','p':'Precision','f':'F1'}

for i in range(len(df_scores)):
  for col_name in ['rouge-1','rouge-2','rouge-l']:
    # print(col_name)
    for key,value in df_scores.loc[i,col_name].items():
      df_scores.loc[i,col_name.upper()+'_'+dict_metric_name[key]] = value 
df_scores

,rouge-1,rouge-2,rouge-l,id,ROUGE-1_F1,ROUGE-1_Precision,ROUGE-1_Recall,ROUGE-2_F1,ROUGE-2_Precision,ROUGE-2_Recall,ROUGE-L_F1,ROUGE-L_Precision,ROUGE-L_Recall,ID_F1,ID_Precision,ID_Recall
0,"{'f': 0.1379310294887041, 'p': 0.1428571428571...","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.14285713785714302, 'p': 0.142857142857...",id_cnn_88698,0.137931,0.142857,0.133333,0,0,0,0.142857,0.142857,0.142857,-,-,-
1,"{'f': 0.25806451175858486, 'p': 0.190476190476...","{'f': 0.13793103020214045, 'p': 0.1, 'r': 0.22...","{'f': 0.27586206444708683, 'p': 0.210526315789...",id_dm_26727,0.258065,0.190476,0.4,0.137931,0.1,0.222222,0.275862,0.210526,0.4,-,-,-
2,"{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.0, 'p': 0.0, 'r': 0.0}",id_dm_205431,0,0,0,0,0,0,0,0,0,-,-,-
3,"{'f': 0.23076922766272187, 'p': 0.142857142857...","{'f': 0.16666666388888893, 'p': 0.1, 'r': 0.5}","{'f': 0.26086956181474485, 'p': 0.166666666666...",id_cnn_47838,0.230769,0.142857,0.6,0.166667,0.1,0.5,0.26087,0.166667,0.6,-,-,-
4,"{'f': 0.6896551677051131, 'p': 0.5555555555555...","{'f': 0.5925925879286695, 'p': 0.4705882352941...","{'f': 0.7142857095153062, 'p': 0.5882352941176...",id_dm_63684,0.689655,0.555556,0.909091,0.592593,0.470588,0.8,0.714286,0.588235,0.909091,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,"{'f': 0.07407406913580279, 'p': 0.066666666666...","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.07407406913580279, 'p': 0.066666666666...",id_dm_183675,0.0740741,0.0666667,0.0833333,0,0,0,0.0740741,0.0666667,0.0833333,-,-,-
9995,"{'f': 0.370370365925926, 'p': 0.27777777777777...","{'f': 0.23999999564800004, 'p': 0.176470588235...","{'f': 0.370370365925926, 'p': 0.27777777777777...",id_dm_14557,0.37037,0.277778,0.555556,0.24,0.176471,0.375,0.37037,0.277778,0.555556,-,-,-
9996,"{'f': 0.4615384570940172, 'p': 0.3461538461538...","{'f': 0.3783783739956173, 'p': 0.28, 'r': 0.58...","{'f': 0.4864864819284149, 'p': 0.375, 'r': 0.6...",id_cnn_90018,0.461538,0.346154,0.692308,0.378378,0.28,0.583333,0.486486,0.375,0.692308,-,-,-
9997,"{'f': 0.30303029825528016, 'p': 0.25, 'r': 0.3...","{'f': 0.12903225331945908, 'p': 0.105263157894...","{'f': 0.2499999951757813, 'p': 0.2105263157894...",id_dm_192809,0.30303,0.25,0.384615,0.129032,0.105263,0.166667,0.25,0.210526,0.307692,-,-,-


In [9]:
def print_scores(df_scores):
  print("Mean scores: ROUGE-1= {}, ROUGE-2= {}, ROUGE-L= {}.".format(df_scores['ROUGE-1_F1'].mean(), df_scores['ROUGE-2_F1'].mean(), df_scores['ROUGE-L_F1'].mean()))
  print("Median scores: ROUGE-1= {}, ROUGE-2= {}, ROUGE-L= {}.".format(df_scores['ROUGE-1_F1'].median(), df_scores['ROUGE-2_F1'].median(), df_scores['ROUGE-L_F1'].median()))
  print("Max scores: ROUGE-1= {}, ROUGE-2= {}, ROUGE-L= {}.".format(df_scores['ROUGE-1_F1'].max(), df_scores['ROUGE-2_F1'].max(), df_scores['ROUGE-L_F1'].max()))
  print("Min scores: ROUGE-1= {}, ROUGE-2= {}, ROUGE-L= {}.".format(df_scores['ROUGE-1_F1'].min(), df_scores['ROUGE-2_F1'].min(), df_scores['ROUGE-L_F1'].min()))


In [10]:
# ROUGE for bart-large-cnn model
print_scores(df_scores)

Mean scores: ROUGE-1= 0.270475114364702, ROUGE-2= 0.12631801472324003, ROUGE-L= 0.25289003591923204.
Median scores: ROUGE-1= 0.23076922579881665, ROUGE-2= 0.06451612466181092, ROUGE-L= 0.19999999555555567.
Max scores: ROUGE-1= 0.999999995, ROUGE-2= 0.999999995, ROUGE-L= 0.999999995.
Min scores: ROUGE-1= 0.0, ROUGE-2= 0.0, ROUGE-L= 0.0.


In [11]:
# loc_save = '/content/drive/My Drive/Colab Notebooks/data'

# df_scores.to_csv(loc_save+'/ROUGE_bart-large-cnn.csv')